# RAG ETL Pipeline для Yandex Handbook

**Архитектура:**
- **Child chunks** (450 токенов) → Qdrant для точного поиска (HYBRID: dense + sparse)
- **Parent chunks** (1500 токенов) → Redis для контекста
- **ParentDocumentRetriever** связывает chunks

**Установка пакетов:**
```bash
uv add langchain-core langchain-community langchain-openai langchain-text-splitters langchain-qdrant
uv add qdrant-client python-dotenv tiktoken tqdm
```

## 1. Переменные окружения

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

# OpenAI настройки
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_api_base = os.getenv('OPENAI_API_BASE')
embedding_model_name = os.getenv('OPENAI_EVBEDDING_MODEL_NAME', 'text-embedding-3-large')

# Qdrant настройки
qdrant_url = os.getenv('QDRANT_URL', 'http://localhost:6333')

# Redis настройки
redis_url = os.getenv('REDIS_URL', 'redis://localhost:6379')

print(" Переменные окружения загружены")
print(f"   • OpenAI API: {'✓' if openai_api_key else '✗'}")
print(f"   • Embedding Model: {embedding_model_name}")
print(f"   • Qdrant URL: {qdrant_url}")
print(f"   • Redis URL: {redis_url}")

 Переменные окружения загружены
   • OpenAI API: ✓
   • Embedding Model: text-embedding-3-large
   • Qdrant URL: http://localhost:6333
   • Redis URL: redis://localhost:6379


## 2. Импорты

In [2]:
import tiktoken
import re
import hashlib
import json
from typing import Dict, Any
from pathlib import Path
from uuid import uuid4

# LangChain
from langchain_core.documents import Document
from langchain_core.load import dumpd, loads
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from langchain_qdrant.fastembed_sparse import FastEmbedSparse
from langchain_community.storage import RedisStore

# Qdrant
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, VectorParams, SparseVectorParams, SparseIndexParams

# Progress bar
from tqdm.auto import tqdm

print(" Импорты загружены")

/home/llm-dev/project/lifelong_learning_assistant/rag/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 Импорты загружены


## 3. Функции очистки текста

In [3]:
def clean_document_text(text: str) -> str:
    """Очистка Markdown-текста от ссылок, файлов, форматирования"""
    if not text:
        return ""
    
    # Удаление ссылок [текст](url) → текст
    text = re.sub(r'\[([^\]]+)\]\([^)]+\)', r'\1', text)
    
    # Удаление изображений ![alt](src)
    text = re.sub(r'!\[[^\]]*\]\([^)]+\)', '', text)
    
    # Удаление URL
    text = re.sub(r'https?://\S+', '', text)
    
    # Удаление форматирования **текст** → текст
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
    text = re.sub(r'\*(.*?)\*', r'\1', text)
    
    # Удаление HTML тегов
    text = re.sub(r'<[^>]+>', '', text)
    
    # Удаление лишних пробелов
    text = re.sub(r'\s{2,}', ' ', text)
    text = text.strip()
    
    # Lowercase для лучшего поиска
    return text.lower()


## 4. Функции для метаданных

In [4]:
def stable_parent_id(meta: Dict[str, Any], text: str) -> str:
    """Детерминированный parent_id"""
    base = (
        str(meta.get("source", "")) + "|" +
        str(meta.get("H1", "")) + "|" + 
        str(meta.get("H2", "")) + "|" + 
        str(meta.get("H3", "")) + "|" +
        (text[:128])
    ).encode("utf-8")
    return hashlib.md5(base).hexdigest()

## 5. Загрузка документов (БЕЗ очистки)

In [5]:
# Настройка токенизатора
encoder = tiktoken.get_encoding("cl100k_base")
tiktoken_len = lambda text: len(encoder.encode(text))

# Путь к данным
DATA_PATH = "/home/llm-dev/project/lifelong_learning_assistant/data/yandex-handbook-downloaded"

# Загружаем документы БЕЗ очистки (нужны заголовки ##, ###)
print(f" Загружаем документы из: {DATA_PATH}")
md_files = list(Path(DATA_PATH).glob("*.md"))
print(f" Найдено {len(md_files)} .md файлов")

documents = []
for file_path in md_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    doc = Document(
        page_content=content,
        metadata={
            'source': str(file_path),
            'filename': file_path.name,
            'total_tokens': tiktoken_len(content)
        }
    )
    documents.append(doc)

print(f"Загружено {len(documents)} документов")

# Статистика
if documents:
    token_counts = [doc.metadata['total_tokens'] for doc in documents]
    print(f"\n Статистика:")
    print(f"   • Средний размер: {sum(token_counts)/len(token_counts):.0f} токенов")
    print(f"   • Минимум: {min(token_counts)} токенов")
    print(f"   • Максимум: {max(token_counts)} токенов")

 Загружаем документы из: /home/llm-dev/project/lifelong_learning_assistant/data/yandex-handbook-downloaded
 Найдено 68 .md файлов
Загружено 68 документов

 Статистика:
   • Средний размер: 9532 токенов
   • Минимум: 873 токенов
   • Максимум: 27611 токенов


## 6. Загрузка index.json для названий документов

In [6]:
# Загружаем index.json
INDEX_PATH = "/home/llm-dev/project/lifelong_learning_assistant/data/yandex-handbook-downloaded/index.json"

with open(INDEX_PATH, 'r', encoding='utf-8') as f:
    index_data = json.load(f)

# Создаем маппинг filename -> title
filename_to_title = {}
for doc_id, doc_info in index_data.items():
    filename = doc_id + '.md'
    title = doc_info.get('title', '')
    filename_to_title[filename] = title

print(f" Загружено {len(filename_to_title)} названий документов")
print(f"\n Примеры:")
for i, (fn, title) in enumerate(list(filename_to_title.items())[:3]):
    print(f"   • {fn} → {title}")

 Загружено 68 названий документов

 Примеры:
   • about.md → Начать учиться
   • pervie-shagi.md → Первые шаги
   • mashinnoye-obucheniye.md → Машинное обучение


## 7. Настройка сплиттеров


In [7]:
# Markdown Header Splitter для parent chunks (используем H1, H2, H3 как в clean_wiki_docs)
headers_to_split_on = [
    ("#", "H1"),
    ("##", "H2"),
    ("###", "H3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,
    strip_headers=False
)

# Recursive splitter для больших parent chunks
parent_recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""],
    keep_separator=True
)

# Child splitter
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=70,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""],
    keep_separator=True
)


## 8. Создание parent chunks

In [8]:
parent_docs = []
total_before_clean = 0
total_after_clean = 0

for doc in documents:
    # Получаем название документа из index.json
    doc_filename = doc.metadata.get('filename', '')
    doc_title = filename_to_title.get(doc_filename, '')
    
    # 1. Разбиваем по заголовкам (ДО очистки!)
    md_chunks = markdown_splitter.split_text(doc.page_content or "")
    
    for chunk in md_chunks:
        # Объединяем метаданные
        merged_meta = {**(doc.metadata or {}), **(chunk.metadata or {})}
        
        # Добавляем название документа
        merged_meta["doc_title"] = doc_title
        
        # Создаем breadcrumbs и path (как в clean_wiki_docs)
        headers = [merged_meta.get(k, "") for k in ("H1", "H2", "H3") if k in merged_meta]
        merged_meta["breadcrumbs"] = " / ".join(headers) if headers else ""
        
        # Path: doc_title / H1 / H2 / H3
        path_parts = [doc_title] if doc_title else []
        path_parts.extend(headers)
        merged_meta["path"] = " / ".join(path_parts) if path_parts else ""
        
        # Level: определяем по наличию заголовков
        merged_meta["level"] = 3 if "H3" in merged_meta else 2 if "H2" in merged_meta else 1 if "H1" in merged_meta else 0
        
        # Сохраняем размер до очистки
        total_before_clean += tiktoken_len(chunk.page_content)
        
        # 2. ТЕПЕРЬ очищаем текст chunk (lowercase и удаление шума)
        chunk.page_content = clean_document_text(chunk.page_content)
        total_after_clean += tiktoken_len(chunk.page_content)
        
        # 3. Создаем parent_id
        merged_meta["parent_id"] = stable_parent_id(merged_meta, chunk.page_content)
        chunk.metadata = merged_meta
        
        # Если чанк слишком большой - разбиваем
        if tiktoken_len(chunk.page_content) > 1500:
            sub_chunks = parent_recursive_splitter.split_documents([chunk])
            for sub in sub_chunks:
                sub.metadata = merged_meta.copy()
                sub.metadata["parent_id"] = stable_parent_id(sub.metadata, sub.page_content)
            parent_docs.extend(sub_chunks)
        else:
            parent_docs.append(chunk)

print(f" Создано {len(parent_docs)} parent chunks")

# Статистика
parent_tokens = [tiktoken_len(doc.page_content) for doc in parent_docs]
levels = [doc.metadata.get('level', 0) for doc in parent_docs]

print(f"\n Статистика parent chunks:")
print(f"   • Среднее: {sum(parent_tokens)/len(parent_tokens):.0f} токенов")
print(f"   • Минимум: {min(parent_tokens)} токенов")
print(f"   • Максимум: {max(parent_tokens)} токенов")
print(f"   • Медиана: {sorted(parent_tokens)[len(parent_tokens)//2]} токенов")

print(f"\n Эффект очистки:")
print(f"   • До очистки: {total_before_clean:,} токенов")
print(f"   • После очистки: {total_after_clean:,} токенов")
print(f"   • Сокращение: {(1 - total_after_clean/total_before_clean)*100:.1f}%")

print(f"\n Распределение по уровням заголовков:")
for i in range(4):
    count = levels.count(i)
    if count > 0:
        level_name = ["Без заголовка", "H1", "H2", "H3"][i]
        print(f"   • {level_name}: {count} chunks ({count/len(parent_docs)*100:.1f}%)")

# Проверяем примеры метаданных
print(f"\n Пример метаданных:")
if parent_docs:
    sample = parent_docs[10]  # Берем 10-й для разнообразия
    print(f"   • doc_title: {sample.metadata.get('doc_title', 'нет')}")
    print(f"   • path: {sample.metadata.get('path', 'нет')}")
    print(f"   • breadcrumbs: {sample.metadata.get('breadcrumbs', 'нет')}")
    print(f"   • level: {sample.metadata.get('level', 0)}")

 Создано 923 parent chunks

 Статистика parent chunks:
   • Среднее: 668 токенов
   • Минимум: 21 токенов
   • Максимум: 1500 токенов
   • Медиана: 584 токенов

 Эффект очистки:
   • До очистки: 653,624 токенов
   • После очистки: 595,848 токенов
   • Сокращение: 8.8%

 Распределение по уровням заголовков:
   • Без заголовка: 50 chunks (5.4%)
   • H2: 401 chunks (43.4%)
   • H3: 472 chunks (51.1%)

 Пример метаданных:
   • doc_title: Байесовский подход к оцениванию
   • path: Байесовский подход к оцениванию / Сопряжённые распределения
   • breadcrumbs: Сопряжённые распределения
   • level: 2


In [9]:
parent_docs[0].metadata

{'source': '/home/llm-dev/project/lifelong_learning_assistant/data/yandex-handbook-downloaded/veroyatnostnyj-podhod-v-ml.md',
 'filename': 'veroyatnostnyj-podhod-v-ml.md',
 'total_tokens': 6115,
 'doc_title': 'Вероятностный подход в ML',
 'breadcrumbs': '',
 'path': 'Вероятностный подход в ML',
 'level': 0,
 'parent_id': '43a09260959c8673a66e2810d418de54'}

## 9. Настройка Qdrant и Redis

In [10]:
# Dense embeddings (OpenAI)
embeddings = OpenAIEmbeddings(
    model=embedding_model_name,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    chunk_size=1000
)

# Sparse embeddings (BM25 для гибридного поиска)
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

print(" Эмбеддинги настроены (dense + sparse)")

# Qdrant
qdrant_client = QdrantClient(host="localhost", port=6333)
collection_name = "yandex_handbook_child_chunks"

try:
    qdrant_client.delete_collection(collection_name)
    print(f"🗑️ Удалена старая коллекция")
except:
    pass

# Создаем коллекцию с поддержкой гибридного поиска
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config={
        "dense": VectorParams(size=3072, distance=Distance.COSINE)
    },
    sparse_vectors_config={
        "sparse": SparseVectorParams(
            index=SparseIndexParams(on_disk=False),
            modifier=models.Modifier.IDF,
        )
    },
)

# Векторное хранилище в гибридном режиме
vectorstore = QdrantVectorStore(
    client=qdrant_client,
    collection_name=collection_name,
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="dense",
    sparse_vector_name="sparse",
)

print(f" Qdrant настроен (HYBRID режим): {collection_name}")

# Redis
parent_store = RedisStore(
    redis_url=redis_url,
    namespace="rag:parents"
)

print(f" Redis настроен: {redis_url}")

Fetching 18 files: 100%|██████████| 18/18 [00:00<00:00, 18.51it/s]


 Эмбеддинги настроены (dense + sparse)
🗑️ Удалена старая коллекция
 Qdrant настроен (HYBRID режим): yandex_handbook_child_chunks
 Redis настроен: redis://localhost:6379


## 10. Загрузка данных в Qdrant и Redis

In [11]:
print(" Загружаем данные...")

# Функции для Redis
def doc_to_bytes(doc: Document) -> bytes:
    as_dict = dumpd(doc)
    return json.dumps(as_dict, ensure_ascii=False).encode("utf-8")

# 1. Загружаем parent chunks в Redis
parent_ids = []
batch = []
for doc in parent_docs:
    pid = doc.metadata["parent_id"]
    batch.append((pid, doc_to_bytes(doc)))
    parent_ids.append(pid)
    
    if len(batch) >= 1000:
        parent_store.mset(batch)
        batch.clear()

if batch:
    parent_store.mset(batch)

print(f" Загружено {len(parent_ids)} parent chunks в Redis")

 Загружаем данные...
 Загружено 923 parent chunks в Redis


In [12]:
# 2. Создаем child chunks и загружаем в Qdrant
child_docs = []
for parent in parent_docs:
    parent_id = parent.metadata["parent_id"]
    children = child_splitter.split_documents([parent])
    
    for child in children:
        # Обновляем метаданные
        child.metadata.update(parent.metadata)
        child.metadata["parent_id"] = parent_id
        child.metadata["child_id"] = str(uuid4())
        child.metadata["is_child_chunk"] = True
        
        # Добавляем path в начало текста для лучшего контекста
        path = child.metadata.get("path", "")
        if path:
            child.page_content = f"{path}\n\n{child.page_content}"
        
        child_docs.append(child)

print(f" Создано {len(child_docs)} child chunks")
print(f"   • Path добавлен в начало каждого child chunk для контекста")

# Загружаем в Qdrant с прогресс-баром
if child_docs:
    batch_size = 100
    print(f"\n Загружаем {len(child_docs)} child chunks в Qdrant...")
    
    for i in tqdm(range(0, len(child_docs), batch_size), desc="Загрузка в Qdrant"):
        batch = child_docs[i:i+batch_size]
        vectorstore.add_documents(batch)
    
    print(f" Child chunks загружены в Qdrant")

# Итоговая статистика
print(f"\n Итоговая статистика:")
print(f"   • Исходных документов: {len(documents)}")
print(f"   • Parent chunks: {len(parent_docs)}")
print(f"   • Child chunks: {len(child_docs)}")
print(f"   • Соотношение child/parent: {len(child_docs)/len(parent_docs):.1f}")

collection_info = qdrant_client.get_collection(collection_name)
print(f"\n Qdrant: {collection_info.points_count:,} векторов")
print(f" Redis: {len(parent_ids):,} parent документов")

# Статистика по child chunks
child_tokens = [tiktoken_len(doc.page_content) for doc in child_docs]
print(f"\n Статистика child chunks:")
print(f"   • Среднее: {sum(child_tokens)/len(child_tokens):.0f} токенов")
print(f"   • Минимум: {min(child_tokens)} токенов")
print(f"   • Максимум: {max(child_tokens)} токенов")
print(f"   • Медиана: {sorted(child_tokens)[len(child_tokens)//2]} токенов")

# Пример child chunk с path
print(f"\n Пример child chunk:")
if child_docs:
    sample = child_docs[5]
    preview = sample.page_content[:300].replace('\n', ' ')
    print(f"   • Превью: {preview}...")

 Создано 2071 child chunks
   • Path добавлен в начало каждого child chunk для контекста

 Загружаем 2071 child chunks в Qdrant...


Загрузка в Qdrant: 100%|██████████| 21/21 [01:44<00:00,  4.98s/it]


 Child chunks загружены в Qdrant

 Итоговая статистика:
   • Исходных документов: 68
   • Parent chunks: 923
   • Child chunks: 2071
   • Соотношение child/parent: 2.2

 Qdrant: 2,071 векторов
 Redis: 923 parent документов

 Статистика child chunks:
   • Среднее: 355 токенов
   • Минимум: 20 токенов
   • Максимум: 515 токенов
   • Медиана: 396 токенов

 Пример child chunk:
   • Превью: Вероятностный подход в ML / Случайность как источник несовершенства модели  сгенерированные из моделей с каждым из типов шума: нормальным, лапласовским и коши. !9 как вы могли заметить, в каждом из подходов после того, как мы зафиксировали признаки (то есть координаты ), остаётся своя степень свобод...


## 11. Тестирование поиска

In [13]:
def bytes_to_doc(b: bytes) -> Document:
    return loads(b.decode("utf-8"))

def search(query: str, k: int = 10):
    """Поиск с Parent-Child Retrieval"""
    # Находим child chunks
    child_results = vectorstore.similarity_search(query, k=k)
    
    # Получаем parent chunks из Redis
    parent_ids = [doc.metadata.get("parent_id") for doc in child_results]
    parent_bytes = parent_store.mget(parent_ids)
    
    # Восстанавливаем parent документы
    results = []
    for pb in parent_bytes:
        if pb:
            parent_doc = bytes_to_doc(pb)
            parent_doc.metadata["chunk_type"] = "parent"
            results.append(parent_doc)
    
    return results

# Тестовые запросы
test_queries = [
    "байесовский подход в машинном обучении",
    "регуляризация в линейной регрессии",
    "апостериорное распределение параметров"
]

print("🔍 Тестируем поиск:\n" + "="*60)

for i, query in enumerate(test_queries, 1):
    print(f"\n{i}. Запрос: '{query}'")
    results = search(query)
    
    if results:
        best = results[0]
        print(f"    Найдено: {len(results)} parent chunks")
        print(f"    Лучший результат:")
        print(f"      • Файл: {best.metadata.get('filename', 'unknown')}")
        print(f"      • Breadcrumbs: {best.metadata.get('breadcrumbs', 'нет')}")
        print(f"      • Размер: {tiktoken_len(best.page_content)} токенов")
        preview = best.page_content[:200].replace('\n', ' ')
        print(f"      • Превью: {preview}...")


🔍 Тестируем поиск:

1. Запрос: 'байесовский подход в машинном обучении'


/tmp/ipykernel_3085/3734101441.py:2: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return loads(b.decode("utf-8"))


    Найдено: 10 parent chunks
    Лучший результат:
      • Файл: bajesovskij-podhod-k-ocenivaniyu.md
      • Breadcrumbs: Байесовский подход и дообучение моделей
      • Размер: 326 токенов
      • Превью: ## байесовский подход и дообучение моделей до сих пор мы в основном рассуждали о моделях машинного обучения как о чём-то, что один раз обучается и дальше навсегда застывает в таком виде, но в жизни та...

2. Запрос: 'регуляризация в линейной регрессии'
    Найдено: 10 parent chunks
    Лучший результат:
      • Файл: tonkosti-obucheniya.md
      • Breadcrumbs: Регуляризация нейронных сетей
      • Размер: 417 токенов
      • Превью: ## регуляризация нейронных сетей смысл термина регуляризация (англ. regularization) гораздо шире привычного вам прибавления - или -нормы вектора весов к функции потерь. фактически он объединяет большо...

3. Запрос: 'апостериорное распределение параметров'
    Найдено: 10 parent chunks
    Лучший результат:
      • Файл: bajesovskij-podhod-k-ocenivaniyu.